In [1]:
import ee
import ee
import geemap
from geemap import *
import json
from geemap import geojson_to_ee, ee_to_geojson
from ipyleaflet import GeoJSON
import os
import ipygee as ui
import datetime
import pandas as pd

#Trigger the authentication flow.
ee.Authenticate()

#Initialize the library.
ee.Initialize()

Enter verification code: 4/1AY0e-g7vIcWmvo2ypgQGGzCZXGxEv5dw0WTvIPx2C9HDCSgW3kXATJDXelI

Successfully saved authorization token.


In [2]:
#initialise map
Map = geemap.Map()

In [19]:
#AOI geometry
AOI_FILE_PATH = os.path.abspath('/Users/joycelynlongdon/Desktop/Cambridge/CambridgeCoding/MRES/Data/GeoJSONS/wwc_concesion.GeoJSON')
OUTPUT_PATH = "/Users/joycelynlongdon/Desktop/Cambridge/CambridgeCoding/MRES/GEE_examples/Output Data"

#visual parameters
VIZ_PARAMS = {'min': 0,'max': 3000,'bands': ['B4', 'B3', 'B2']} #RGB Composite 

#satellite data source
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')

#set point for map centering
point = ee.Geometry.Point(18.4276, -2.6357)



In [4]:
#define area of interest
def open_aoi_json(aoi_file_path):
    with open(aoi_file_path) as f:
        studyRegion = json.load(f)
    studyRegion = ee.FeatureCollection(studyRegion).first().geometry()
    #print(aoi)
    return studyRegion

studyRegion = open_aoi_json(AOI_FILE_PATH)

In [5]:
#define cloud mask
def cloudMask(image):
  #Bits 3 and 5 are cloud shadow and cloud, respectively.
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    #Get the pixel QA band.
    qa = image.select('pixel_qa')
    #Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    return image.updateMask(mask)

In [6]:
#reduce image and clip to aoi
def clip_image_to_aoi(image):
    #reduce image collection into one multiband image taking the median value of each pixel
    image = image.median()
    #clip image to required boundary for studyRegion
    return image.clip(aoi) #NEED TO FIGURE OUT HOW TO DO THIS SEAMLESSLY WITH THE VARYING AOI'S

In [40]:
#filter temporally at period of one month
Jan = l8_spatial.filterDate('2013-01-01','2013-01-31')
Feb = l8_spatial.filterDate('2013-02-01','2013-02-28')
Mar = l8_spatial.filterDate('2013-03-01','2013-03-31')
Apr = l8_spatial.filterDate('2013-04-01','2013-04-30')
May = l8_spatial.filterDate('2013-05-01','2013-05-30')
Jun = l8_spatial.filterDate('2013-06-01','2013-06-30')
Jul = l8_spatial.filterDate('2013-07-01','2013-07-31')
Aug = l8_spatial.filterDate('2013-08-01','2013-08-31')
Sept = l8_spatial.filterDate('2013-09-01','2013-09-30')
Oct = l8_spatial.filterDate('2013-10-01','2013-10-31')
Nov = l8_spatial.filterDate('2013-11-01','2013-11-30')
Dec = l8_spatial.filterDate('2013-12-01','2013-12-31')

In [41]:
#cloud mask data
Jan_masked = Jan.map(cloudMask)
Feb_masked = Feb.map(cloudMask)
Mar_masked = Mar.map(cloudMask)
Apr_masked = Apr.map(cloudMask)
May_masked = May.map(cloudMask)
Jun_masked = Jun.map(cloudMask)
Jul_masked = Jul.map(cloudMask)
Aug_masked = Aug.map(cloudMask)
Sept_masked = Sept.map(cloudMask)
Oct_masked = Oct.map(cloudMask)
Nov_masked = Nov.map(cloudMask)
Dec_masked = Dec.map(cloudMask)

In [42]:
#create median composite and select bands
Jan_med = Jan_masked.median().select(['B2','B3','B4','B5','B6','B7','B10','B11'])
Feb_med = Feb_masked.median().select(['B2','B3','B4','B5','B6','B7','B10','B11'])
Mar_med = Mar_masked.median().select(['B2','B3','B4','B5','B6','B7','B10','B11'])
Apr_med= Apr_masked.median().select(['B2','B3','B4','B5','B6','B7','B10','B11'])
May_med= May_masked.median().select(['B2','B3','B4','B5','B6','B7','B10','B11'])
Jun_med= Jun_masked.median().select(['B2','B3','B4','B5','B6','B7','B10','B11'])
Jul_med = Jul_masked.median().select(['B2','B3','B4','B5','B6','B7','B10','B11'])
Aug_med = Aug_masked.median().select(['B2','B3','B4','B5','B6','B7','B10','B11'])
Sept_med = Sept_masked.median().select(['B2','B3','B4','B5','B6','B7','B10','B11'])
Oct_med= Oct_masked.median().select(['B2','B3','B4','B5','B6','B7','B10','B11'])
Nov_med = Nov_masked.median().select(['B2','B3','B4','B5','B6','B7','B10','B11'])
Dec_med= Dec.median().select(['B2','B3','B4','B5','B6','B7','B10','B11'])

In [43]:
#clip to study region
Jan_image = Jan_med.clip(studyRegion)#clip to studyRegion
Feb_image = Feb_med.clip(studyRegion)#clip to studyRegion
Mar_image = Mar_med.clip(studyRegion)#clip to studyRegion
Apr_image = Apr_med.clip(studyRegion)#clip to studyRegion
May_image = May_med.clip(studyRegion)#clip to studyRegion
Jun_image = Jun_med.clip(studyRegion)#clip to studyRegion
Jul_image = Jul_med.clip(studyRegion)#clip to studyRegion
Aug_image = Aug_med.clip(studyRegion)#clip to studyRegion
Sept_image = Sept_med.clip(studyRegion)#clip to studyRegion
Oct_image = Oct_med.clip(studyRegion)#clip to studyRegion
Nov_image = Nov_med.clip(studyRegion)#clip to studyRegion
Dec_image = Dec_med.clip(studyRegion)#clip to studyRegion

In [52]:
#check out and visualise imagery
Map.addLayer(_image, VIZ_PARAMS)
Map

Map(bottom=66445.83203125, center=[-1.6747151822261304, 18.34530115127564], controls=(WidgetControl(options=['…

In [56]:
#export June
geemap.ee_export_image(Jun_image, filename='wwc_concession_Jun_2013.tif', scale=30, region=studyRegion, file_per_band=False)

Generating URL ...
An error occurred while downloading.
Total request size (414021120 bytes) must be less than or equal to 33554432 bytes.


In [57]:
#export June
geemap.ee_export_image_to_drive(Jun_image, description='wwc_concession_Jun_2013', folder='wwc_concession_data', region=studyRegion, scale=30)


Exporting wwc_concession_Jun_2013 ...


In [ ]:
#export July
geemap.ee_export_image_to_drive(Jul_image, filename=filename, folder='wwc_concession_data', region=studyRegion, scale=30, file_per_band=False)


In [ ]:
#export August
geemap.ee_export_image_to_drive(Aug_image, description='wwc_concession_Jun_2013', folder='wwc_concession_data', region=studyRegion, scale=30)


In [ ]:
#export September
geemap.ee_export_image_to_drive(Jun_image, description='wwc_concession_Jun_2013', folder='wwc_concession_data', region=studyRegion, scale=30)
